In [ ]:
# Import necessary libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os, sys
# Add the 'scripts' directory to the Python path for module imports
sys.path.append(os.path.abspath(os.path.join('..', 'scripts')))

# Set max rows and columns to display
pd.set_option('display.max_columns', 200)
pd.set_option('display.max_rows', 200)

# Configure logging
from logger import SetupLogger
# Assuming this class is defined in scripts/
from data_preprocessor import DataPreprocessor  

logger = SetupLogger(log_file='../logs/notebooks.log').get_logger()

# Set figure size
plt.figure(figsize=(12, 5))

In [ ]:
# Set google drive url for loading the dataset
data = '../data/BrentOilPrices.csv'

# Setup the data preprocessor class
processor = DataPreprocessor(data, logger=logger)
# Load the data
price_data = processor.load_data()